# Couette flow simulations

In [1]:
%matplotlib widget 
# global import
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# local import
import lppydsmc as ld

## Case 1 

In [2]:
path_couette_1 = "/home/calot/Documents/projets/lppydsmc/benchmarks/config_files/couette_1.ini"

In [3]:
# launching simulations
simulation = True
if(simulation):
    ld.main(path_couette_1, save = True);

|    it    |   INIT   |  INJECT  |   DEL    | C. WALLS | C. PARTS |  REACT   | EXEC TIME (s) |
----------------------------------------------------------------------------------------------
|    0     |   4000   |    NA    |    0     |    0     |    0     |    NA    |       0       |


ValueError: operands could not be broadcast together with shapes (812,3) (2,) (812,3) 

### Loading Results

In [4]:
results_path = "/home/calot/Documents/projets/lppydsmc/benchmarks/results/couette_1/monitoring.h5" # seems ok ...
store = pd.HDFStore(results_path)
print(store.keys())

['/dsmc_collisions', '/dsmc_tracking', '/particles', '/wall_collisions']


In [164]:
h = 1e-3
l = 2.5e-4
dy = h/40.
dx = 2.5e-4/10.
U = 100 # m/s
dt = 1e-7

In [8]:
df_particles = store['particles']

In [13]:
df_particles['v2'] = df_particles['vx']*df_particles['vx']+df_particles['vy']*df_particles['vy']+df_particles['vz']*df_particles['vz']
df_particles['v'] = np.sqrt(df_particles['v2'])

In [40]:
frames = 0.2 # taking the last 80% of the simulation
max_frame = np.max(df_particles.index)
df_partial = df_particles.loc[df_particles.index > frames*max_frame]

### Scatter plot

In [176]:
df_last_frame = df_partial.loc[df_partial.index == max_frame]
fig, ax = plt.subplots(constrained_layout=True)
ax.axis('equal')
sns.scatterplot(data = df_last_frame, x='x',y='y')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='x', ylabel='y'>

### Distribution

In [170]:
bins = 100

fig, ax = plt.subplots(2,2, constrained_layout = True)
ax[0,0].hist(df_partial['vx'], bins = bins, color = 'r', density = False)
ax[0,1].hist(df_partial['vy'], bins = bins, color = 'g', density = False)
ax[1,0].hist(df_partial['vz'], bins = bins, color = 'b', density = False)
ax[1,1].hist(df_partial['v'], bins = bins, color = 'k', density = False);

ax[0,0].set_xlabel('vx (m/s)')
ax[0,1].set_xlabel('vy (m/s)')
ax[1,0].set_xlabel('vz (m/s)')
ax[1,1].set_xlabel('v (m/s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'v (m/s)')

### Distribution $v_x$ depending on $y$

In [37]:
df_particles['vx/U'] = df_particles['vx']/U
df_particles['y/h'] = df_particles['y']/h

In [42]:
fig, ax = plt.subplots(constrained_layout = True)
sns.histplot(data = df_partial, x = 'vx/U', y = 'y/h', cbar = True, bins = 40, stat = 'density', ax = ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='vx/U', ylabel='y/h'>

### Profile - and dependance on the x-position

In [ ]:
df_particles['y_int'] = (df_particles['y']/dy).astype(int) 

In [149]:
def plot_profile(ax, df, height, lenght, dx, dy, wall_speed, factor = 0):
    # df['y_int'] = (df['y']/dy).astype(int) 
    if(factor == 0):
        results = df.groupby('y_int').mean()['vx']
    else:
        df_partial_x_low = df.loc[df['x']<factor*dx]
        df_partial_x_mid = df.loc[(df['x']<lenght-factor*dx) & (df_partial['x']>factor*dx)]
        df_partial_x_high = df.loc[df['x']>lenght-factor*dx]
        results_low = df_partial_x_low.groupby('y_int').mean()['vx']
        results_mid = df_partial_x_mid.groupby('y_int').mean()['vx']
        results_high = df_partial_x_high.groupby('y_int').mean()['vx']
        
    ax.set_xlabel('vx/U')
    ax.set_ylabel('y/h')
    if(factor == 0):
        ax.plot(results.values/wall_speed, results.index*dy/height, color = 'k',  label = 'average')
    else:
        ax.plot(results_low.values/wall_speed, results_low.index*dy/height, color = 'r', label = 'x < {:.3}'.format(factor*dx/lenght))
        ax.plot(results_mid.values/wall_speed, results_mid.index*dy/height, color = 'g',  label = '{:.3} < x < {:.3}'.format(factor*dx/lenght, 1-factor*dx/lenght))
        ax.plot(results_high.values/wall_speed, results_high.index*dy/height, color = 'b',  label = '{:.3} < x'.format(1-factor*dx/lenght))
    ax.legend(loc = 'lower right')
    return ax

In [151]:
fig, ax = plt.subplots(constrained_layout = True)
plot_profile(ax, df_partial, height = h, lenght = l, dx = dx, dy = dy, wall_speed = U, factor = 3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='vx/U', ylabel='y/h'>

#### Prolonging profile near the top wall

In [89]:
def prolong_profile(profile, values, height):
    dy = values[1]-values[0]
    first_der = (profile[1:]-profile[:profile.shape[0]-1])/dy
    second_der = (first_der[1:]-first_der[:first_der.shape[0]-1])/dy
    
    dy_end = (height-values[-1])
    prolong_first_order = profile[-1]+dy_end*first_der[-1]
    prolong_second_order = prolong_first_order+0.5*dy_end*dy_end*second_der[-1]
    
    return values[-1]+dy_end, prolong_first_order, prolong_second_order

In [121]:
print(prolong_profile(results_mid.values,  results_mid.index.values*dy, h)) # y value, first prolong, 2nd one

(0.001, 65.87592781197878, 69.84733577991821)


### Dependance of the profile on time

We may not be in steady flow yet.

In [166]:
def plot_profile_evolution(ax, df, height, lenght, dx, dy, wall_speed, time_step, number_of_splits = 1):
    s = df.shape[0]
    number_of_frames_per_curve = s//number_of_splits
    max_index = np.max(df.index.values)
    number_of_true_frames = max_index//number_of_splits
    for k in range(number_of_splits):
        df_ = df[k*number_of_frames_per_curve:(k+1)*number_of_frames_per_curve]
        results = df_.groupby('y_int').mean()['vx']
        ax.plot(results.values/wall_speed, results.index*dy/height,  label = '{:.1e} < t < {:.1e} s'.format(time_step*k*number_of_true_frames, time_step*(k+1)*number_of_true_frames))

    ax.set_xlabel('vx/U')
    ax.set_ylabel('y/h')
    ax.legend(loc = 'lower right')


In [167]:
fig, ax = plt.subplots(constrained_layout = True)
plot_profile_evolution(ax, df_particles, height = h, lenght = l, dx = dx, dy = dy,  wall_speed = U, time_step = dt, number_of_splits = 3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …